# 🌍 TranslateGemma 12B - Dịch thuật AI

Model dịch thuật mạnh mẽ của Google hỗ trợ 55 ngôn ngữ

**Lưu ý:** Chọn GPU runtime (Runtime > Change runtime type > T4 GPU)

## 📦 Bước 1: Cài đặt thư viện

In [ ]:
!pip install -q transformers>=4.46.0 torch pillow accelerate sentencepiece protobuf
print("✅ Đã cài đặt xong!")

## 🔄 Bước 2: Load model (chạy 1 lần, mất ~1-2 phút)

In [ ]:
import torch
from transformers import AutoModelForImageTextToText, AutoProcessor

print("🔄 Đang load TranslateGemma 12B...")
print("⏳ Vui lòng đợi 1-2 phút...")

model_id = "google/translategemma-12b-it"
processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

print("✅ Model 12B đã load xong! Sẵn sàng dịch thuật.")

## 🛠️ Bước 3: Hàm dịch thuật

In [ ]:
def translate_text(text, source_lang="vi", target_lang="en", max_tokens=200):
    """
    Dịch văn bản
    
    Args:
        text: Văn bản cần dịch
        source_lang: Mã ngôn ngữ nguồn (vi, en, zh, ja, ko, fr, de, es, th, id, cs...)
        target_lang: Mã ngôn ngữ đích
        max_tokens: Số token tối đa cho output
    """
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "source_lang_code": source_lang,
                    "target_lang_code": target_lang,
                    "text": text,
                }
            ],
        }
    ]
    
    inputs = processor.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_dict=True,
        return_tensors="pt"
    ).to(model.device, dtype=torch.bfloat16)
    
    input_len = len(inputs['input_ids'][0])
    
    with torch.inference_mode():
        generation = model.generate(
            **inputs,
            do_sample=False,
            max_new_tokens=max_tokens
        )
    
    generation = generation[0][input_len:]
    translated = processor.decode(generation, skip_special_tokens=True)
    
    return translated


def translate_image(image_url, source_lang="en", target_lang="vi", max_tokens=200):
    """
    Trích xuất và dịch text từ ảnh
    
    Args:
        image_url: URL của ảnh
        source_lang: Mã ngôn ngữ nguồn
        target_lang: Mã ngôn ngữ đích
        max_tokens: Số token tối đa cho output
    """
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source_lang_code": source_lang,
                    "target_lang_code": target_lang,
                    "url": image_url,
                },
            ],
        }
    ]
    
    inputs = processor.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_dict=True,
        return_tensors="pt"
    ).to(model.device, dtype=torch.bfloat16)
    
    input_len = len(inputs['input_ids'][0])
    
    with torch.inference_mode():
        generation = model.generate(
            **inputs,
            do_sample=False,
            max_new_tokens=max_tokens
        )
    
    generation = generation[0][input_len:]
    translated = processor.decode(generation, skip_special_tokens=True)
    
    return translated

print("✅ Hàm dịch thuật đã sẵn sàng!")

## 🧪 Bước 4: Test dịch thuật

### Dịch văn bản

In [ ]:
# Test 1: Tiếng Việt -> Tiếng Anh
text_vi = "Xin chào! Tôi là trợ lý AI. Hôm nay trời đẹp quá!"
print(f"🇻🇳 Tiếng Việt: {text_vi}")

result = translate_text(text_vi, source_lang="vi", target_lang="en")
print(f"🇺🇸 Tiếng Anh: {result}")

In [ ]:
# Test 2: Tiếng Anh -> Tiếng Việt
text_en = "Hello! I am an AI assistant. The weather is beautiful today!"
print(f"🇺🇸 Tiếng Anh: {text_en}")

result = translate_text(text_en, source_lang="en", target_lang="vi")
print(f"🇻🇳 Tiếng Việt: {result}")

In [ ]:
# Test 3: Tiếng Việt -> Tiếng Trung
text_vi = "Tôi yêu học tiếng Trung. Nó rất thú vị!"
print(f"🇻🇳 Tiếng Việt: {text_vi}")

result = translate_text(text_vi, source_lang="vi", target_lang="zh")
print(f"🇨🇳 Tiếng Trung: {result}")

### Dịch từ ảnh

In [ ]:
# Test 4: Trích xuất và dịch text từ ảnh
image_url = "https://c7.alamy.com/comp/2YAX36N/traffic-signs-in-czech-republic-pedestrian-zone-2YAX36N.jpg"
print(f"🖼️  Ảnh: {image_url}")

result = translate_image(image_url, source_lang="cs", target_lang="vi")
print(f"🇻🇳 Dịch sang Tiếng Việt: {result}")

## 🎯 Bước 5: Dịch văn bản của bạn

Thay đổi text và ngôn ngữ theo ý muốn!

In [ ]:
# Nhập văn bản của bạn
my_text = "Hôm nay tôi học lập trình Python"  # Thay đổi text này
source = "vi"  # Ngôn ngữ nguồn
target = "en"  # Ngôn ngữ đích

print(f"📝 Input ({source}): {my_text}")
result = translate_text(my_text, source_lang=source, target_lang=target)
print(f"✅ Output ({target}): {result}")

## 📚 Ngôn ngữ được hỗ trợ

TranslateGemma hỗ trợ 55 ngôn ngữ. Một số mã phổ biến:

- `vi` - Tiếng Việt 🇻🇳
- `en` - Tiếng Anh 🇺🇸
- `zh` - Tiếng Trung 🇨🇳
- `ja` - Tiếng Nhật 🇯🇵
- `ko` - Tiếng Hàn 🇰🇷
- `fr` - Tiếng Pháp 🇫🇷
- `de` - Tiếng Đức 🇩🇪
- `es` - Tiếng Tây Ban Nha 🇪🇸
- `th` - Tiếng Thái 🇹🇭
- `id` - Tiếng Indonesia 🇮🇩
- `cs` - Tiếng Séc 🇨🇿
- `ru` - Tiếng Nga 🇷🇺
- `ar` - Tiếng Ả Rập 🇸🇦
- `pt` - Tiếng Bồ Đào Nha 🇵🇹
- `it` - Tiếng Ý 🇮🇹

Và nhiều ngôn ngữ khác!